In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat


## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('maperiod:%2d, %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

## Walkforward

In [ ]:
class DecisiveAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.maperiod = self.strategy.params.maperiod
        
    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0

    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):
        self.ntrade += 1
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        
        outp = {
            'params': (self.maperiod),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade
        }
        
        return outp
    

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    for res in result:
        r = res[0].analyzers.decisive.get_analysis()
        params.append(r['params'])
        n_trades.append(r['ntrade'])
        profit.append(r['profit'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_trades = n_trades[prof_ind]
    
    print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_trades))
    return (best_params, best_profit, best_trades)

In [ ]:
datapath = os.path.join('../data/raw/demo/orcl-1995-2014.txt')

periods = [
{'oos': 2003, 'is': (2000, 2002)},
{'oos': 2004, 'is': (2001, 2003)},
{'oos': 2005, 'is': (2002, 2004)},]

best_oos_params = {}
for oos_is in periods:
    start_year, end_year = oos_is['is']
    print('START/END YEAR: {}/{}'.format(start_year, end_year))
    cerebro = bt.Cerebro()

    strats = cerebro.optstrategy(
        TestStrategy,
        maperiod=range(10, 31, 5))
    
    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.optreturn = False
    cerebro.broker.setcash(1000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.YahooFinanceCSVData(dataname=datapath,
                                        fromdate=datetime.datetime(start_year, 1, 1),
                                        todate=datetime.datetime(end_year, 1, 1),
                                        reverse=False)
    cerebro.adddata(data)

    result = cerebro.run(maxcpus=1)
    best_param, _, _ = best_result_from_cerebro_opti_run(result)
    
    best_oos_params[oos_is['oos']] = best_param

print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))

In [ ]:
print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))